<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo2/2_dvc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=10mKgunAZowpvpttdYdjoDeCM2RSMGXer" width="100%">

# Versionamiento de Datos
---

En este notebook daremos una introducción práctica al versionamiento de datos con la herramienta `dvc`, para ello, debemos instalarla en el entorno de Google Colaboratory. Recuerde que se trata de un sistema basado en el sistema operativo Linux, más específicamente la distribución Ubuntu:

In [ ]:
!cat /etc/os-release

Podemos instalar `dvc` con el siguiente comando:

In [ ]:
!pip install dvc dvc-gdrive

Adicionalmente, instalaremos algunas herramientas auxiliares:

In [ ]:
!apt install tree

Validamos que la herramienta se encuentra instalada:

In [ ]:
!dvc --version

Finalmente, importamos las librerías necesarias:

In [ ]:
import os
from IPython import get_ipython

# Data Version Control
---

Data Version Control (DVC) es un sistema de control de versiones específico para datos, similar a Git, pero diseñado específicamente para manejar grandes conjuntos de datos y modelos. Con DVC, se pueden rastrear y controlar las versiones de los datos utilizados en un proyecto de aprendizaje automático (machine learning), así como los cambios realizados en los modelos y scripts de entrenamiento.

<center><img src="https://drive.google.com/uc?export=view&id=1jZi-b90okFLR-ercTxcdNM9GYDqfWMKj" width="80%"></center>

Algunas de las funciones principales de DVC incluyen:

- **Rastreo de cambios en los datos y los modelos**: DVC permite rastrear los cambios realizados en los datos y los modelos y revertirlos si es necesario.
- **Almacenamiento de datos**: DVC permite almacenar grandes conjuntos de datos en un sistema de almacenamiento externo, como Amazon S3, y hacer referencia a ellos en el repositorio DVC.
- **Replicabilidad**: DVC permite reproducir fácilmente los resultados de un proyecto de aprendizaje automático, ya que registra todos los cambios realizados en los datos y los modelos.
- **Colaboración**: DVC permite que varios desarrolladores trabajen usando exactamente los mismos datos, características y modelos.

En este notebook veremos los tres principales usos de `dvc` para versionamiento de archivos:

## **1. Versionamiento de Datos**
---

Una de las principales desventajas de sistemas de versionamiento como Git es que no tienen la capacidad de manejar volúmenes grandes de archivos. De hecho, se recomienda que un repositorio no tenga más de 1 GB de tamaño.

No obstante, en proyectos de machine learning es muy común el uso y la generación de datos, por lo cual, es necesario buscar una alternativa que nos permita mantener un registro de la información que estamos manejando. **Data Version Control** (DVC) surge como una herramienta que busca extender Git para hacer seguimiento de datos, toda la parte de versionamiento se sigue manejando desde Git, pero los archivos grandes se ubican en algún lugar de almacenamiento distinto del mismo repositorio.

<center><img src="https://drive.google.com/uc?export=view&id=1SVHchK8RXUOOApwxrVPn10C3K02jTnk7" width="80%"></center>

## **2. Creación del Repositorio**
---

DVC se usa de una forma muy equivalente a Git, es decir, necesitamos mantener un repositorio de código para versionar los datos también, por ello, vamos a crear un repositorio:

In [ ]:
!mkdir myrepo
%cd myrepo

Recuerde configurar su identidad en los siguientes campos:

In [ ]:
!git config --global user.email "ejemplo@unal.edu.co"
!git config --global user.name "Mi nombre o username"
!git config --global init.defaultBranch master

Inicializamos el repositorio:

In [ ]:
!git init

Para iniciar con el versionamiento de datos, vamos a crear la carpeta `data` dentro del repositorio:

In [ ]:
!mkdir data

## **3. Conjunto de Datos**
---

Ahora, vamos a descargar dos archivos planos que conforman un dataset de nombres en inglés [Names Corpus](https://www.kaggle.com/datasets/nltkdata/names):

In [ ]:
!wget https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/master/u2/female.txt -O data/female.txt
!wget https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/master/u2/male.txt -O data/male.txt

Veamos la estructura de directorios que tenemos ahora:

In [ ]:
!tree

También podemos ver el contenido de alguno de los archivos:

In [ ]:
!head data/male.txt

## **4. Agregando Datos a DVC**
---

Como estamos trabajando con Git, estos archivos se mantendrán bajo el seguimiento de esta herramienta. Podemos validarlo:

In [ ]:
!git status

Como podemos ver, toda la carpeta `data` se encuentra en el área de trabajo. Vamos a comenzar e integrar `dvc` para que veamos cómo se simplifica el manejo de Git. Para ello, inicializamos el proyecto con `dvc init`:

In [ ]:
!dvc init

Veamos qué cambió en el repositorio:

In [ ]:
!ls -a

Como podemos ver, se creó la carpeta `.dvc` (almacenamiento de la herramienta) y un archivo `.dvcignore` (qué datos debe ignorar la herramienta).

Vamos a registrar los datos para su seguimiento con `dvc add`:

In [ ]:
!dvc add data/male.txt data/female.txt

Veamos qué cambios tiene la carpeta `data` luego de agregar archivos en `dvc`:

In [ ]:
!ls -a data/

Como podemos ver, se crearon tres archivos:

- `male.txt.dvc` y `female.txt.dvc`: metadatos que se conservarán en el repositorio de Git y enlazan a los datos originales.
- `.gitignore`: específica a Git que no debe mantener registro de los datos originales, únicamente de los metadatos.

Podemos inspeccionar el contenido de estos archivos, primero, los metadatos:

In [ ]:
!cat data/female.txt.dvc

Como podemos ver, se almacena un código hash que identifica la versión del archivo (similar a los commits), su tamaño y el nombre del archivo al que hace referencia.

<center><img src="https://drive.google.com/uc?export=view&id=1I_85fV6wp5o-2F0UTEvOD2YUcx1EKrZw" width="82%"></center>

También podemos ver el archivo `.gitignore`:

In [ ]:
!cat data/.gitignore

Podemos ver que se excluyen los archivos que corresponden directamente a los datos. Con esto, podemos pasar a agregar al área de preparación los archivos necesarios de `dvc`:

In [ ]:
!git add data/.gitignore data/*.txt.dvc

Podemos validar el estado de los archivos en Git:

In [ ]:
!git status

Procedemos a generar un commit:

In [ ]:
!git commit -m "Inicializamos dvc y agregamos el dataset de nombres"

## **5. Almacenamiento Remoto de Datos**
---

Al igual que con Github, podemos usar distintas plataformas de almacenamiento para guardar de forma remota los datos. En este caso utilizaremos _Google Drive_, para poder conectarnos debemos obtener un archivo de autenticación. Comenzamos ingresando a [este enlace](https://console.cloud.google.com/) donde deberá indicar el país donde se encuentra y aceptar los términos y condiciones:

<img src="https://drive.google.com/uc?export=view&id=1CR2uLHNpOJnxm-me2t1NnxWQFQvl7dOi" width="80%">

Ahora, debe dar click sobre el botón `Seleccionar Proyecto`:

<img src="https://drive.google.com/uc?export=view&id=1uwGJ4QelpUUXXw5plkKtvYBc_I6hncWU" width="80%">

Y debe dar click sobre `Nuevo Proyecto`:

<img src="https://drive.google.com/uc?export=view&id=174eNXXOwLzHm21gNCvb1OZmXQ_JjWhpJ" width="80%">

Ahora deberá asignarle un nombre al proyecto, se sugiere que sea `mlds6`:

<img src="https://drive.google.com/uc?export=view&id=1dmUqgDXyGcPCaVPDkxRysj-AdYH4wdj6" width="80%">

Debe esperar un tiempo hasta que el proyecto se cree, seguido aparecerá una notificación en la parte superior derecha donde podrá seleccionar el proyecto recién creado:

<img src="https://drive.google.com/uc?export=view&id=131wawVvjX_V1twWztrgg_Na8D5EsdZW0" width="80%">

Ahora, debemos habilitar el uso de _Google Drive_ en el proyecto recién creado, para esto deberá ingresar en el buscador el siguiente texto `google drive api` y dar click sobre la opción con el mismo nombre:

<img src="https://drive.google.com/uc?export=view&id=1DXNL3SvDuaTaauncocE3g3oQkoED9jQL" width="80%">

Esto le permitirá habilitarla. Tambien puedes ir a el [panel de APIs](https://console.cloud.google.com/apis/dashboard) para hacer el mismo proceso:

<img src="https://drive.google.com/uc?export=view&id=1i4EmAfR8XfFL8ag6axIbUJy9-n9HsvPc" width="80%">

Ahora, debe dirigirse a la parte de credenciales dentro del API de _Google Drive_:

<img src="https://drive.google.com/uc?export=view&id=1ou52BmExwMhMo5JHLIjq2tdI-HWjzxWa" width="80%">

Debe dar click sobre `Crear Credenciales` y seleccionar la opción de cuenta de servicio. En la primera parte se le solicitará un nombre para la cuenta de servicio, se recomienda que utilice `mlds6`:

<img src="https://drive.google.com/uc?export=view&id=1K4MYFTXizfsh3sRt__Bx7Ht37ZAJm1Mt" width="80%">

> **Nota**: debajo del id que se genera automáticamente para la cuenta de servicio también aparece un correo electrónico, deberá copiar este correo para más adelante.

En la segunda parte deberá agregar permisos para la cuenta de servicio, se recomienda que seleccione la opción de `Propietario`:

<img src="https://drive.google.com/uc?export=view&id=12yTAfRmwGUUsZQ13-6xC5eyebx_vYlq_" width="80%">

En la tercera parte no es necesario ingresar ningún campo, así que puede darle en continuar directamente.

Ahora, debemos dirigirnos a la parte superior izquierda y dar click sobre el menú general de la consola de Google (el que tiene tres lineas horizontales) y debemos dar click sobre `IAM y Administración`:

<img src="https://drive.google.com/uc?export=view&id=1l_E9p4_8W-F8eKX9vFOg34Uq0zRo6OXG" width="80%">

En la parte de la izquierda debe seleccionar `Cuentas de Servicio`, y aquí debe aparecer la cuenta que se acabó de crear. Debe dar click sobre los tres puntos en la parte de acciones de la cuenta de servicio y seleccionar la opción `Administrar Claves`:

<img src="https://drive.google.com/uc?export=view&id=16brbwtGPfFYCGxh6jqRBR6UdZ-_x19RR" width="80%">

Ahora debe dar click en `Crear clave nueva` y descargar las credenciales en formato `json`:

<img src="https://drive.google.com/uc?export=view&id=1DAsbmdZyNUvBtkwozPoRQynYqWCmvJA9" width="80%">

**Nota**: esto descargará un archivo en formato `json` en su computador, debe renombrarlo a `credentials.json` y subirlo a _Google Colab_.

Por último, vamos a crear una carpeta en drive y a copiar su id:

> Nota: Tenga en cuenta que si utiliza una cuenta institucional, puede que no tenga permisos para compartir recuersos a cuentas externas, funcionalidad que será necesaria más adelante

<img src="https://drive.google.com/uc?export=view&id=18hgHr_-Jr4o3YVAp08vhtBeIIVFP65LU" width="80%">

Deberá reemplazar el id en la siguiente variable:

In [ ]:
drive_id = "..." # reemplace aquí el id de su carpeta
os.environ["DRIVEID"] = drive_id

Para enlazar `dvc` con drive debemos agregarlo como remoto:

In [ ]:
!dvc remote add -d storage "gdrive://$DRIVEID"

Para que `dvc` pueda escribir sobre esta carpeta debemos compartirla con permisos de edición al correo de la cuenta de servicio que creamos anteriormente. En la carpeta de drive, damos compartir y copiamos la dirección creada:

<img src="https://drive.google.com/uc?export=view&id=1J6t4m81xM166pPcSUjCCIEa0eGqqrZTB" width="80%">

Y configuramos `dvc` para que se autentique en _Google Drive_ con esta cuenta de servicio:

In [ ]:
!dvc remote modify storage gdrive_use_service_account true

Creamos una variable de entorno para que `dvc` sepa dónde encontrar las credenciales de la cuenta de servicio:

In [ ]:
import json
with open("../credentials.json") as f:
    os.environ["GDRIVE_CREDENTIALS_DATA"] = f.read()

Guardamos en Git los cambios de DVC:

In [ ]:
!git add .dvc/config
!git commit -m "Agregamos drive remoto a dvc"

Para subir los datos a drive, usamos el comando `dvc push`:

In [ ]:
!dvc push

De esta forma, tendremos los datos almacenados en drive (se guardan de forma compacta, así que probablemente no los pueda leer directamente en drive).

Veamos cómo podemos descargar los datos, primero eliminamos el caché de dvc y los mismos datos:

In [ ]:
!rm -rf .dvc/cache
!rm data/male.txt data/female.txt

Ahora, con el comando `dvc pull` podemos descargar los datos:

In [ ]:
!dvc pull

Podemos validar que los archivos se encuentran nuevamente en el repositorio:

In [ ]:
!tree

Por último veamos qué pasa cuando realizamos cambios en los datos, vamos a duplicar el contenido del archivo `male.txt`. Primero veamos su tamaño original:

In [ ]:
!ls -sh data/male.txt | awk '{ print $1 }'

Ahora, lo duplicamos:

In [ ]:
!cp data/male.txt tmp.txt
!cat tmp.txt tmp.txt > data/male.txt
!rm tmp.txt

Veamos el tamaño del nuevo archivo:

In [ ]:
!ls -sh data/male.txt | awk '{ print $1 }'

Procedemos a actualizar el registro de este archivo:

In [ ]:
!dvc add data/male.txt

Nuevamente, debemos actualizar los cambios en Git y generar un commit:

In [ ]:
!git add .dvc/config data/male.txt.dvc
!git commit -m "Cambios en male.txt"

Subimos la nueva versión de los datos:

In [ ]:
!dvc push

## **6. Creación y Recuperación de Versiones**
---

Si deseamos volver a una versión específica de los datos, es suficiente con ir al commit de Git donde se usaron. Veamos los commits que tenemos:

In [ ]:
!git log

> **Nota**: el código hash será completamente diferente para usted, así que lo vamos a extraer buscando el mensaje del primer commit que realizamos.

In [ ]:
command = """
git --no-pager log --decorate=short --pretty=oneline |\
        grep -P 'Inicializamos' |\
        awk '{ print $1 }'
"""
commit = (
        get_ipython()
        .getoutput(command)[0]
        )
os.environ["COMMIT"] = commit
print(commit)

Regresamos el repositorio a la versión inicial:

In [ ]:
!git checkout "$COMMIT"

Para recuperar los datos en la versión de este commit, usamos `dvc checkout`:

In [ ]:
!dvc checkout

Podemos validar que el archivo se encuentre en su tamaño original:

In [ ]:
!ls -sh data/male.txt | awk '{ print $1 }'

Con esto podemos ver que DVC es una herramienta muy útil para aplicaciones relacionadas con datos y siguiendo la misma filosofía que Git.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [DVC](https://dvc.org/)
- [DVC: Data Versioning](https://dvc.org/doc/start/data-management/data-versioning)
- [DVC: Data and Model Access](https://dvc.org/doc/start/data-management/data-and-model-access)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*